Importing the necessary packages

In [2]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine 

Import DB connection details from external file (not synced to repository) to avoid sharing connection details. Alternatively define a variable for the connection details for the database directly here.

In [3]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the connection string from environment variables
conn_string = os.getenv("POSTGRESQL_URL")
# conn_string='postgresql://user@host:port/db-name'

Establishing the database connection

In [5]:
 
db = create_engine(conn_string) 
conn = db.connect() 


Load csv file

In [105]:
df = pd.read_csv('grammarchecks.csv')

Prepare lists determining which columns should be contained in which table

In [33]:
languages_headers = ['language_id','lang_name', 'glottocode', 'walscode', 'iso639_3', 'lat', 
                     'lon', 'coord_notes', 'area_wals', 'area_glottolog', 'sort_affiliation', 'genus_wals', 'family_genus']
core_props_headers = ['language_id','constituent_order','constituent_order_wals','adposition_order','adposition_order_wals',
                      'genitive_order','genitive_order_wals','demonstrative_order', 'demonstrative_order_wals','definite_article','article_distinct_third',
                      'demonstrative_as_third','nominal_person','apc_direction','bound_person_direction','person_allowed',
                      'person_third_available','number_allowed','ppdc']
article_props_headers = ['language_id','article_order','demonstrative_w_article','nominal_person_w_article']


Check that the right (sub)dataframes are picked out

In [24]:
df[languages_headers]

,language_id,lang_name,glottocode,walscode,iso639_3,lat,lon,coord_notes,area_wals,area_glottolog,sort_affiliation,genus_wals,family_genus
0,1,Japanese,nucl1643,jpn,jpn,35.000000,135.000000,NaN,Eurasia,Eurasia,(Altaic?) Isolate,Japanese,Japonic
1,2,Korean,kore1280,kor,kor,37.500000,128.000000,NaN,Eurasia,Eurasia,(Altaic?) Isolate,Korean,Koreanic
2,3,Evenki,even1259,eve,evn,61.972000,94.689000,NaN,Eurasia,Eurasia,(Altaic?) Tungusic,Tungusic,Tungusic
3,4,Turkish,nucl1301,tur,tur,39.866700,32.866700,NaN,Eurasia,Eurasia,(Altaic?) Turkic,Turkic,Turkic
4,5,Mangarrayi,mang1381,myi,mpc,-14.800000,133.500000,"glottolog 4.1 longitude shifted, use glottolog...",Australia,Australia,"(Australian) Mangarrayi-Maran, Mangarrayi",Mangarrayi,"Mangarrayi-Maran, Mangarrayi"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,130,Mandarin,mand1415,mnd,cmn,40.020900,116.228000,NaN,Eurasia,Eurasia,"Sino-Tibetan, Sinitic",Chinese,"Sino-Tibetan, Sinitic"
130,131,Lakkia,lakk1238,lkk,lbc,24.116900,110.106000,NaN,Eurasia,Eurasia,"Tai-Kadai, Kam-Tai, Lakkia-Biao",Kadai,"Tai-Kadai, Kam-Tai"
131,132,Hungarian,hung1274,hun,hun,46.906859,19.655527,NaN,Eurasia,Eurasia,"Uralic, Ugric",Ugric,"Uralic, Ugric"
132,133,Finnish,finn1318,fin,fin,64.762800,25.557700,NaN,Eurasia,Eurasia,"Uralic, Finnic",Finnic,"Uralic, Finnic"


In [13]:
df[core_props_headers]

,language_id,constituent_order,constituent_order_wals,adposition_order,adposition_order_wals,genitive_order,genitive_order_wals,demonstrative_order,demonstrative_order_wals,definite_article,article_distinct_third,demonstrative_as_third,nominal_person,apc_direction,bound_person_direction,person_allowed,person_third_available,number_allowed,ppdc
0,1,OV,OV,post,post,GenN,GenN,DemN,DemN,n,n,n,y,pre,n,all,y,non-sg,y
1,2,OV,OV,post,post,GenN,GenN,DemN,DemN,n,n,n,y,pre,n,all,y,non-sg,y
2,3,OV,OV,post,post,GenN,GenN,DemN,DemN,n,n,n,y,pre,n,all,y,all,NaN
3,4,OV,OV,post,post,GenN,GenN,DemN,DemN,n,n,y,y,pre,n,all,y,non-sg,NaN
4,5,OV,OV,pre,pre,NoDom,NoDom,DemN,DemN,n,n,y,y,pre,n,all,y,non-sg,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,130,VO,VO,NoDom,NoDom,GenN,GenN,DemN,DemN,n,n,n,y,pre,n,all,y,non-sg,y
130,131,VO,NaN,pre,NaN,GenN,NaN,NDem,NaN,n,n,n,y,pre,n,NaN,?,NaN,NaN
131,132,VO,VO,post,post,GenN,GenN,DemN,DemN,y,y,n,y,pre,n,no 3,n,non-sg,NaN
132,133,VO,VO,post,post,GenN,GenN,DemN,DemN,n (but sm case marking effects),n,n,y,pre,n,all,y,non-sg,NaN


Article properties are only relevant for languages with articles, so there are a lot of NA values. We are getting rid of any rows that contain three (or more) NA values (because we have three contentful columns). I'm avoiding a threshold of 1 because the columns concerning combinability of articles with demonstratives and nominal person are not established in as much detail in the current dataset.

In [15]:
df[article_props_headers].dropna(thresh=3)

,language_id,article_order,demonstrative_w_article,nominal_person_w_article
19,20,NArt,"n (demonstrative modifier), y („appositive“ de...",y
26,27,ArtN,y,unclear
35,36,NArt,y,unclear
36,37,NArt,NaN,y
38,39,NArt,y,y
40,41,NArt,"n (or y, depending on analysis of demonstrative)",y
41,42,NArt,y? (no example given though),y
44,45,NArt,y (optional),y
45,46,NArt,y (optional),y
48,49,ArtN,y (def agr),y


In [ ]:
df[languages_headers].to_sql('languages', con=conn, if_exists='append',  index=False) 

134

In [36]:
df[core_props_headers].to_sql('core_properties', con=conn, if_exists='append',  index=False) 


134

In [29]:
df[article_props_headers].dropna(thresh=3).to_sql('article_properties', con=conn, if_exists='append',  index=False) 

48

# Prepare the datasources table

Create a copy of the dataframe containing only the language_id and refs columns.
Creating a copy avoids problems when dropping columns and replacing empty fields later on.


In [121]:
references = df[['language_id','refs']].copy()

Create a new column where the reference strings are split into lists of references (delimiter is the semicolon). Then drop the original refs column as it's no longer needed.

In [122]:
references['splitrefs'] = references['refs'].str.split(';')
references.drop(columns='refs',inplace=True)


Ensure that fields containing no reference list (i.e. because there was no reference and there is NULL/na value instead) get replaced by an empty list to ensure proper behaviour later on. 

In [123]:
references.loc[:,'splitrefs'] = references['splitrefs'].apply(lambda x: x if isinstance(x, list) else [])
print(references)

     language_id                                          splitrefs
0              1  [Hinds 1988:254, 261,  Noguchi 1997:780,  Furu...
1              2          [Sohn 1994:284, 292,  Choi 2014: 151-154]
2              3                          [Nedjalkov 1997:197, 199]
3              4                         [Kornfilt 1997:288, 297f.]
4              5                     [Merlan 1989 [1982]: 103, 203]
..           ...                                                ...
129          130                   [Boskovic & Hsieh 2013:sec. 7.3]
130          131                                     [Fan 2019:137]
131          132  [Kenesei,Vago & Fenyvesi 1998:269,  Höhn 2016:...
132          133                   [Sulkala & Karjalainen 1992:277]
133          134     [Andrews 1975:192-194,  Andrews 2003:ch. 17.3]

[134 rows x 2 columns]


Explode the splitrefs field

In [124]:
ref_exploded = references.explode('splitrefs').reset_index(drop=True)
print(ref_exploded)


     language_id                         splitrefs
0              1               Hinds 1988:254, 261
1              1                  Noguchi 1997:780
2              1              Furuya 2008:sec. 3.2
3              1                      Inokuma 2009
4              1                    (Coulmas 1982)
..           ...                               ...
167          132  Kenesei,Vago & Fenyvesi 1998:269
168          132                     Höhn 2016:559
169          133    Sulkala & Karjalainen 1992:277
170          134              Andrews 1975:192-194
171          134             Andrews 2003:ch. 17.3

[172 rows x 2 columns]


In [126]:
ref_exploded['ref_short'] = ref_exploded['splitrefs'].str.split(':').str[0].str.lstrip()
ref_exploded['ref_pages'] = ref_exploded['splitrefs'].str.split(':').str[1].str.lstrip()

Drop unnecessary splitrefs column and check output

In [127]:
ref_exploded.drop(columns='splitrefs', inplace=True)
print(ref_exploded)

     language_id                     ref_short ref_pages
0              1                    Hinds 1988  254, 261
1              1                  Noguchi 1997       780
2              1                   Furuya 2008  sec. 3.2
3              1                  Inokuma 2009       NaN
4              1                (Coulmas 1982)       NaN
..           ...                           ...       ...
167          132  Kenesei,Vago & Fenyvesi 1998       269
168          132                     Höhn 2016       559
169          133    Sulkala & Karjalainen 1992       277
170          134                  Andrews 1975   192-194
171          134                  Andrews 2003  ch. 17.3

[172 rows x 3 columns]


Enter dataframe into SQL table

In [128]:
ref_exploded.to_sql('datasources', con=conn, if_exists='replace',  index=False) 


172

Since we used "if_exists='replace'" a previously defined FOREIGN KEY relation does not persist and we need to (re-)establish it with the following commands, which first create a cursor and use that to execute an SQL command to add the FOREIGN KEY constraint to link the datasources table to the languages table.

In [133]:
conn = psycopg2.connect(conn_string 
                        ) 
conn.autocommit = True
cursor = conn.cursor() 

cursor.execute('ALTER TABLE public.datasources ADD CONSTRAINT datasources_languages_fk FOREIGN KEY (language_id) REFERENCES public.languages(language_id) ON DELETE CASCADE;')

Close connection to database

In [6]:
conn.close()
